In [21]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import numpy as np
import re
import json

In [12]:
def get_buy_webpage(city, 
                    state, 
                    pg_num, 
                    htype=[
                        'house',
                        'multi-family',
                    ]):
    
    overhead = 'https://www.trulia.com'
    dangle = 'for_sale'

    city = city.title()\
               .replace(' ', '_')
    state = state.upper()

    dict_alias = {
        'house': 'SINGLE-FAMILY_HOME',
        'condo': 'APARTMENT,CONDO,COOP',
        'townhouse': 'TOWNHOUSE',
        'multi-family': 'MULTI-FAMILY',
        'land': 'LOT%7CLAND',
        'mobile/manufactured': 'MOBILE%7CMANUFACTURED',
        'other': 'UNKNOWN',
    }

    aliases = [dict_alias[h] for h in htype]
    houses = ','.join(aliases)

    webpage = f'{overhead}/{dangle}/{city},{state}/{houses}_type/{pg_num}_p/'
    return webpage


In [13]:
url = get_buy_webpage('philadelphia', 'pa', 1)

In [14]:
def get_buy_apt_urls_per_page(city,
                              state,
                              pg_num,
                              htype=['house', 
                                     'multi-family']):

    webpage = get_buy_webpage(city, state, pg_num, htype)
    
    # Here we added User-Agent to the header of our request 
    # It is because sometimes the web server will check the
    # different fields of the header to block robot scrapers
    # User-Agent is the most common one because it is specific 
    # to your browser.
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_3) \
            AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36'}
    response = requests.get(webpage, headers=headers)
    results = response.content

    if not response.status_code == 404:
        soup = BeautifulSoup(results, 'lxml')
        
        apt_class = 'PropertyCard__PropertyCardContainer-sc-1ush98q-0 gsDQZj Box-sc-8ox7qa-0 jIGxjA'
        apt_tags = soup.find_all('div', class_=apt_class)
        
        apt_link_tags = [tag.find('a') for tag in apt_tags]
        apt_urls = [tag['href'] for tag in apt_link_tags]
    
    return apt_urls
        

In [15]:
apt_urls_pg1 = get_buy_apt_urls_per_page('philadelphia', 'pa', 1)
apt_urls_pg1

['/p/pa/philadelphia/1311-foulkrod-st-philadelphia-pa-19124--2017208781',
 '/p/pa/philadelphia/302-carpenter-ln-philadelphia-pa-19119--1037425672',
 '/p/pa/philadelphia/3409-hess-st-philadelphia-pa-19136--2017291136',
 '/p/pa/philadelphia/6523-lincoln-dr-philadelphia-pa-19119--2017190538',
 '/p/pa/philadelphia/6838-ogontz-ave-philadelphia-pa-19138--2017307773',
 '/p/pa/philadelphia/1017-surrey-rd-philadelphia-pa-19115--2017166645',
 '/p/pa/philadelphia/8444-frankford-ave-philadelphia-pa-19136--2017298714',
 '/p/pa/philadelphia/102-w-apsley-st-philadelphia-pa-19144--1139698957',
 '/p/pa/philadelphia/5823-upland-way-philadelphia-pa-19131--2017263482',
 '/p/pa/philadelphia/11610-bustleton-ave-philadelphia-pa-19116--2017179901',
 '/p/pa/philadelphia/124-e-duval-st-philadelphia-pa-19144--2090061622',
 '/p/pa/philadelphia/2411-w-sergeant-st-philadelphia-pa-19132--2089969379',
 '/p/pa/philadelphia/411-e-willow-grove-ave-philadelphia-pa-19118--2089898696',
 '/p/pa/philadelphia/1910-rittenhouse

In [16]:
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_3) \
            AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36'}
apt_wb = 'https://www.trulia.com/p/pa/philadelphia/302-carpenter-ln-philadelphia-pa-19119--1037425672'
response = requests.get(apt_wb, headers=headers)
results = response.content
soup = BeautifulSoup(results, 'lxml')

In [28]:
jfile = soup.find('script', attrs={
    'id': '__NEXT_DATA__',
    'type': 'application/json',
}).get_text()


In [37]:
jdict = json.loads(jfile)
for key in jdict.keys():
    print(key)

dataManager
props
page
query
buildId
assetPrefix
runtimeConfig


In [59]:
jdict['props'].keys()

dict_keys(['abExperiments', 'abTracking', 'session', '_page', 'asPath', 'homeDetails', 'viewer', 'search', 'homeDetailsPromise', 'apolloState', 'apolloHeaders'])

In [66]:
jdict['props']['homeDetails'].keys()

dict_keys(['url', 'media', 'metadata', 'pageText', 'currentStatus', '__typename', 'tracking', 'primaryNavigation', 'secondaryNavigation', 'isSaveable', 'isShareable', 'preferences', 'location', 'adTargetings', 'price', 'heroTags', 'priceChange', 'bedrooms', 'bathrooms', 'floorSpace', 'provider', 'activeForRentListing', 'surroundings', 'hoaFee', 'mortgageInfo', 'nearbyHomes', 'features', 'publicRecord', 'comparables', 'description', 'titleToPriceHistory', 'priceHistory', 'localProtections', 'taxes', 'marketComparisons', 'breadcrumbNavigation', 'backToSearch', 'localInfoSummary', 'parcel', 'nearbyPointsOfInterest', 'seoDescription', 'similarHomes', 'foreclosureInfo', 'community', 'activeForSaleListing', 'assignedSchools', 'providerListingId', 'flagListingReportTypes', 'affordability', 'propertyType'])

In [70]:
jdict['props']['homeDetails']['media'].keys()

dict_keys(['metaTagHeroImages', '__typename', 'topThirdHeroImages', 'totalPhotoCount', 'threeDHomes', 'mapWithPin', 'videos', 'photos', 'streetView'])

In [76]:
jdict['props']['homeDetails']['media']['photos'][2]

{'url': {'extraSmallSrc': 'https://static.trulia-cdn.com/pictures/thumbs_5/zillowstatic/ISb91c89ycbre71000000000.jpg',
  'smallSrc': 'https://static.trulia-cdn.com/pictures/thumbs_6/zillowstatic/ISb91c89ycbre71000000000.jpg',
  'mediumSrc': 'https://static.trulia-cdn.com/pictures/thumbs_6/zillowstatic/ISb91c89ycbre71000000000.jpg',
  'largeSrc': 'https://static.trulia-cdn.com/pictures/thumbs_6/zillowstatic/ISb91c89ycbre71000000000.jpg',
  'hiDpiExtraSmallSrc': 'https://static.trulia-cdn.com/pictures/thumbs_6/zillowstatic/ISb91c89ycbre71000000000.jpg',
  'hiDpiSmallSrc': 'https://static.trulia-cdn.com/pictures/thumbs_6/zillowstatic/ISb91c89ycbre71000000000.jpg',
  'hiDpiMediumSrc': 'https://static.trulia-cdn.com/pictures/thumbs_6/zillowstatic/ISb91c89ycbre71000000000.jpg',
  'hiDpiLargeSrc': 'https://static.trulia-cdn.com/pictures/thumbs_6/zillowstatic/ISb91c89ycbre71000000000.jpg',
  '__typename': 'MEDIA_ImageResource'},
 'webpUrl': {'extraSmallWebpSrc': 'https://static.trulia-cdn.com/

In [80]:
image_url = 'https://static.trulia-cdn.com/pictures/thumbs_5/zillowstatic/ISb91c89ycbre71000000000.jpg'
img_data = requests.get(image_url).content
with open('image_name.jpg', 'wb') as handler:
    handler.write(img_data)

In [78]:
local_file = open('local_image.jpg', 'wb')
resp.raw.decode_content = True


<Response [200]>